In [1]:
import requests
import gradio as gr
from PIL import Image, ImageDraw, ImageFont
 
def request_vision(image_path):
    features = "SmartCrops"
    endpoint = f"https://fimtrus-computervision2.cognitiveservices.azure.com/computervision/imageanalysis:analyze?api-version=2024-02-01&features={features}&smartcrops-aspect-ratios=1,1.1,1.5"
    api_key = "ee6d3d3de333480291b572fc7147b015"
 
 
    headers = {
        "Content-Type": "application/octet-stream",
        "Ocp-Apim-Subscription-Key": api_key
    }
 
    with open(image_path, 'rb') as image:
        image_data = image.read()
 
    response = requests.post(endpoint,
                            headers=headers,
                            data=image_data)
 
    response_json = response.json()
    print(response_json)
 
    return response_json
 
def change_image(image_path):
 
    if image_path is None:
        return None
    
    image = Image.open(image_path)
    draw = ImageDraw.Draw(image)
    try:
        font = ImageFont.truetype("NanumGothic.ttf", 30)
    except IOError:
        # Arial 폰트가 없으면 기본 폰트를 사용합니다.
        font = ImageFont.load_default()
    
    # request_vision을 통해 데이터를 받아 온다.
    # 받은 데이터를 파싱해서 image에 그린다.
 
    response_json = request_vision(image_path)
    # 받은 데이터를 파싱해서 image에 그린다.  
    if "objectsResult" in response_json and "values" in response_json["objectsResult"]:  
        for obj in response_json["objectsResult"]["values"]:  
            box = obj["boundingBox"]  
            x, y, w, h = box["x"], box["y"], box["w"], box["h"]  
            print(x, y, w, h)
            draw.rectangle([(x, y), (x + w, y + h)], outline="red", width=2)  
            if "tags" in obj:  
                for tag in obj["tags"]:  
                    draw.text((x, y - 10), tag["name"], fill="red", font=font)  
 
    if "tagsResult" in response_json and "values" in response_json["tagsResult"]:  
        for obj in response_json["tagsResult"]["values"]:  
            print(obj['name'], int(obj['confidence']*100))
 
    if "smartCropsResult" in response_json and "values" in response_json["smartCropsResult"]:  
        for obj in response_json["smartCropsResult"]["values"]:  
            box = obj["boundingBox"]  
            x, y, w, h = box["x"], box["y"], box["w"], box["h"]  
            print(x, y, w, h)
            draw.rectangle([(x, y), (x + w, y + h)], outline="red", width=2)  
            draw.text((x, y - 10), str(obj['aspectRatio']), fill="red", font=font)  
 
 
    return image
 
with gr.Blocks() as demo:
 
    input_image = gr.Image(label="이미지 선택", type="filepath", width=800)
    output_image = gr.Image(label="출력 이미지", type="pil", width=800, interactive=False)
 
    input_image.change(fn=change_image, inputs=[input_image], outputs=[output_image])
 
demo.launch()
 

c:\Users\USER\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


{'modelVersion': '2023-10-01', 'metadata': {'width': 1024, 'height': 1024}, 'smartCropsResult': {'values': [{'aspectRatio': 1.0, 'boundingBox': {'x': 0, 'y': 0, 'w': 1020, 'h': 1020}}, {'aspectRatio': 1.1, 'boundingBox': {'x': 0, 'y': 24, 'w': 1020, 'h': 927}}, {'aspectRatio': 1.5, 'boundingBox': {'x': 0, 'y': 120, 'w': 1020, 'h': 679}}]}}
0 0 1020 1020
0 24 1020 927
0 120 1020 679
{'modelVersion': '2023-10-01', 'metadata': {'width': 802, 'height': 538}, 'smartCropsResult': {'values': [{'aspectRatio': 1.0, 'boundingBox': {'x': 163, 'y': 0, 'w': 511, 'h': 511}}, {'aspectRatio': 1.1, 'boundingBox': {'x': 138, 'y': 0, 'w': 552, 'h': 502}}, {'aspectRatio': 1.5, 'boundingBox': {'x': 0, 'y': 0, 'w': 750, 'h': 499}}]}}
163 0 511 511
138 0 552 502
0 0 750 499


In [ ]:
headers = {
    "Content-Type": "application/octet-stream",
    "Ocp-Apim-Subscription-Key": api_key
}

params = {
    "features": "objects",
    "api-version": "2024-02-01"
}

def draw_rectangle(image_path):
    # Image_path를 받아와서 객체화 시킨다.
    image = Image.open(image_path)
    draw = ImageDraw.Draw(image)
    # 이미지 데이터를 서버로 request 한다.
    with open(image_path, "rb") as image_file:
        image_data = image_file.read()
    response = requests.post(
        endpoint,
        headers=headers,
        params=params,
        data=image_data  # Assuming the API accepts a URL
    )
    response_json = response.json()
    # 받아몬 데이터를 파성해서 이미지 위메 그려준다.
    object_detection_list = response_json['objectsResult']['values']
    for c in object_detection_list:
        bounding_box = c['boundingBox']
        tag_list = c['tags']
        x, y, w, h = bounding_box['x'], bounding_box['y'], bounding_box['w'], bounding_box['h']
        # Draw rectangle
        draw.rectangle([(x, y), (x + w, y + h)], outline="red", width=2)
        # Draw label
